In [ ]:
import numpy as np
import pandas as pd
import requests
import math 
from scipy import stats
from statistics import mean
import xlsxwriter

from secret import IEX_CLOUD_API_TOKEN

In [ ]:
stock_tickers = pd.read_csv('Data/sp_500_stocks.csv')
stock_tickers

In [ ]:
symbol = 'AAPL'
api_base_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
get_response = requests.get(api_base_url)
stock_data = get_response.json()
stock_data['year1ChangePercent']

In [ ]:
def chunks(series, n):
    for i in range(0, len(series), n):
        yield series[i:i + n]

symbol_sublists = list(chunks(stock_tickers['Ticker'], 100))
symbol_sublists_tickers = []
for i in range(0, len(symbol_sublists)):
    symbol_sublists_tickers.append(','.join(symbol_sublists[i]))

Dataframe_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
Dataframe = pd.DataFrame(columns = Dataframe_columns)
for symbol in symbol_sublists_tickers:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    stock_data = requests.get(batch_api_call_url).json()
    for stock_ticker in symbol.split(','):
        # Function 1
        Dataframe = Dataframe.append(
            pd.Series (
                [
                    stock_ticker,
                    stock_data[stock_ticker]['price'],
                    stock_data[stock_ticker]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = Dataframe_columns
            ),
            ignore_index = True
        )

Dataframe

In [ ]:
# Function 2
Dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
Dataframe = Dataframe[:50]
Dataframe.reset_index(inplace = True)
Dataframe

In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    try:
        float(portfolio_size)
    except ValueError:
        print('Please enter an integer')
        portfolio_size = input('Enter the size of your portfolio: ')

portfolio_input()

In [ ]:
position_size = float(portfolio_size) / len(Dataframe.index)
for i in range(0, len(Dataframe.index)):
    Dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / Dataframe.loc[i, 'Price'])
Dataframe

In [ ]:
high_quality_momentum_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

high_quality_momentum_dataframe = pd.DataFrame(columns = high_quality_momentum_columns)

for symbol in symbol_sublists_tickers:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    stock_data = requests.get(batch_api_call_url).json()
    for stock_ticker in symbol.split(','):
        high_quality_momentum_dataframe = high_quality_momentum_dataframe.append(
            pd.Series (
                [
                    stock_ticker,
                    stock_data[stock_ticker]['price'],
                    'N/A',
                    stock_data[stock_ticker]['stats']['year1ChangePercent'],
                    'N/A',
                    stock_data[stock_ticker]['stats']['month6ChangePercent'],
                    'N/A',
                    stock_data[stock_ticker]['stats']['month3ChangePercent'],
                    'N/A',
                    stock_data[stock_ticker]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = high_quality_momentum_columns
            ),
            ignore_index = True
        )
    

In [ ]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in high_quality_momentum_dataframe.index:
    for time_period in time_periods:
        high_quality_momentum_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(high_quality_momentum_dataframe[f'{time_period} Price Return'], high_quality_momentum_dataframe.loc[row, f'{time_period} Price Return']) / 100

high_quality_momentum_dataframe

In [ ]:
for row in high_quality_momentum_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(high_quality_momentum_dataframe.loc[row, f'{time_period} Return Percentile'])
    high_quality_momentum_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

high_quality_momentum_dataframe.sort_values('HQM Score', ascending = False, inplace = True)

high_quality_momentum_dataframe

In [ ]:
high_quality_momentum_dataframe = high_quality_momentum_dataframe[:50]
high_quality_momentum_dataframe.reset_index(drop = True, inplace = True)
high_quality_momentum_dataframe

In [ ]:
portfolio_input()

position_size = float(portfolio_size) / len(high_quality_momentum_dataframe.index)
for i in range(0, len(high_quality_momentum_dataframe.index)):
    high_quality_momentum_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / high_quality_momentum_dataframe.loc[i, 'Price'])
high_quality_momentum_dataframe

In [ ]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
high_quality_momentum_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format':'0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['One-Year Price Return', percent_format],
    'E': ['One-Year Return Percentile', percent_format],
    'F': ['Six-Month Price Return', percent_format],
    'G': ['Six-Month Return Percentile', percent_format],
    'H': ['Three-Month Price Return', percent_format],
    'I': ['Three-Month Return Percentile', percent_format],
    'J': ['One-Month Price Return', percent_format],
    'K': ['One-Month Return Percentile', percent_format],
    'L': ['HQM Score', percent_format]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_format)


In [ ]:
writer.save()